## 1. Naive-Bayes


A continuación se realiza la validación de los dos conjuntos de datos utilizando tanto validación cruzada como simple y tanto con corrección de Laplace como sin ella.

In [1]:
from Datos import Datos
from Clasificador import ClasificadorNaiveBayes
from EstrategiaParticionado import ValidacionCruzada, ValidacionSimple
import pandas as pd

dataset_german = Datos('data/german.data')
dataset_tic = Datos('data/tic-tac-toe.data')

cruzada = ValidacionCruzada(5)
simple = ValidacionSimple(0.2, 5)
seed = 29

datasets = [dataset_tic, dataset_german]
particionados = [simple, cruzada]

resultados = []

for p in particionados:
    for l in [False, True]:
        resultado_parcial = []
        for data in datasets:
            error, std = ClasificadorNaiveBayes(l).validacion(p, data, seed)
            resultado_parcial.append('{:.6f} +/- {:.6f}'.format(error, std))
        resultados.append(resultado_parcial)

Una vez realizada la validación, se muestran los datos agrupados en una misma tabla.

In [2]:
df = pd.DataFrame(resultados, 
                  columns=['Error tic-tac-toe.data','Error german.data'], 
                  index=pd.MultiIndex.from_tuples([('Simple', 'No'), 
                                                   ('Simple', 'Si'), 
                                                   ('Cruzada', 'No'), 
                                                   ('Cruzada', 'Si')], 
                                                  names=['Validación', 'Laplace']))

df

Error tic-tac-toe.data      Error german.data
Validación Laplace                                              
Simple     No       0.293194 +/- 0.023647  0.261000 +/- 0.018276
           Si       0.293194 +/- 0.023647  0.264000 +/- 0.015297
Cruzada    No       0.293297 +/- 0.031973  0.248000 +/- 0.019647
           Si       0.291214 +/- 0.030275  0.252000 +/- 0.024413

Para ambos conjuntos de datos, el error obtenido es similar. Se obtiene un error ligeramente menor al validar los datos de `german.data`. 

En cuanto a los distintos tipos de validación (simple o cruzada), al aplicarlos en `tic-tac-toe.data`, la diferencia es apenas notable. Al contrario, en `german.data`, la validación cruzada obtiene un error menor que la validación simple (con una diferencia de aproximadante 2%). 

La corrección de Laplace tampoco parece que tenga demasiado impacto en la validación de ambos conjuntos de datos (la única situación en la que parece afectar es en `german.data` cuando se utiliza validación cruzada, pero aún así es solo un cambio de aproximadamente un 1%). Esto podría sugerir que para ambos conjuntos de datos, el impacto de la corrección de Laplace es pequeño debido a que no hay ninguna tabla de atributo con una frecuencia de 0.  

## 2. Scikit-Learn

Lo primero de todo es separar los datos de los conjuntos de datos en lo que comúnmente se denomina X e y (los valores de los atributos, y los valores de la clase).

In [3]:
from sklearn import naive_bayes
from sklearn import preprocessing
from sklearn import model_selection
import numpy as np

x_tic = dataset_tic.datos.to_numpy()[:, :-1]
y_tic = dataset_tic.datos.to_numpy()[:, -1]
x_ger = dataset_german.datos.to_numpy()[:, :-1]
y_ger = dataset_german.datos.to_numpy()[:, -1]
xys = [(x_tic, y_tic), (x_ger, y_ger)]

A continuación realizamos la validación para MultinomialNB.

In [4]:
sk_simple = model_selection.ShuffleSplit(n_splits=5, test_size=0.2)
sk_cruzada = model_selection.KFold(n_splits=5, shuffle=True)
sk_particiones = [sk_simple, sk_cruzada]

resultados_multinomial = []

for particion in sk_particiones:
    for l in [1.0e-10, 1.0]:
        resultado_parcial = []
        for x, y in xys:
            scores = []
            for train_index, test_index in particion.split(x):
                # MultinomialNB
                model = naive_bayes.MultinomialNB(alpha=l)
                model.fit(x[train_index], y[train_index])
                
                scores.append(model.score(x[test_index], y[test_index]))

            resultado_parcial.append('{:.6f} +/- {:.6f}'.format(1 - np.mean(scores), np.std(scores)))
        resultados_multinomial.append(resultado_parcial)


Realizamos la validación para GaussianNB (en este caso no tiene sentido aplicar Laplace ya que se modela cada atributo como una distribución normal).

In [5]:
resultados_gaussian = []

for particion in sk_particiones:
    resultado_parcial = []
    for x, y in xys:
        scores = []
        for train_index, test_index in particion.split(x):
            model = naive_bayes.GaussianNB()
            model.fit(x[train_index], y[train_index])
            
            scores.append(model.score(x[test_index], y[test_index]))

        resultado_parcial.append('{:.6f} +/- {:.6f}'.format(1 - np.mean(scores), np.std(scores)))
    resultados_gaussian.append(resultado_parcial)

Realizamos la validación para CategoricalNB.

In [11]:
resultados_categorical = []

for particion in sk_particiones:
    for l in [1.0e-10, 1.0]:
        index = -1
        resultado_parcial = []
        for x, y in xys:
            index += 1
            scores = []
            for train_index, test_index in particion.split(x):
                model = naive_bayes.CategoricalNB(alpha=l)
                model.fit(x[train_index], y[train_index])
                
                try:
                    scores.append(model.score(x[test_index], y[test_index]))
                except IndexError as e:
                    print(f'Error en iteración con {["tic-tac-toe.data", "german.data"][index]}. Dato en test no apareció en train, por lo tanto el clasificador no tiene una probabilidad asociada al valor.')

            resultado_parcial.append('{:.6f} +/- {:.6f}'.format(1 - np.mean(scores), np.std(scores)))
        resultados_categorical.append(resultado_parcial)

Error en iteración con german.data. Dato en test no apareció en train, por lo tanto el clasificador no tiene una probabilidad asociada al valor.
Error en iteración con german.data. Dato en test no apareció en train, por lo tanto el clasificador no tiene una probabilidad asociada al valor.
Error en iteración con german.data. Dato en test no apareció en train, por lo tanto el clasificador no tiene una probabilidad asociada al valor.
Error en iteración con german.data. Dato en test no apareció en train, por lo tanto el clasificador no tiene una probabilidad asociada al valor.
Error en iteración con german.data. Dato en test no apareció en train, por lo tanto el clasificador no tiene una probabilidad asociada al valor.


Por último, aplicamos OneHotEncoder a los conjuntos de datos para comprobar como afecta a MultinomialNB.

In [19]:
enc = preprocessing.OneHotEncoder()

# Transformación de los datos
x_tic_enc = enc.fit_transform(x_tic)
x_ger_enc = enc.fit_transform(x_ger)

xys_enc = [(x_tic_enc, y_tic), (x_ger_enc, y_ger)]

resultados_multinomial_encoded = []

for particion in sk_particiones:
    for l in [1.0e-10, 1.0]:
        resultado_parcial = []
        for x, y in xys_enc:
            scores = []
            for train_index, test_index in particion.split(x):
                # MultinomialNB
                model = naive_bayes.MultinomialNB(alpha=l)
                model.fit(x[train_index], y[train_index])
                
                scores.append(model.score(x[test_index], y[test_index]))

            resultado_parcial.append('{:.6f} +/- {:.6f}'.format(1 - np.mean(scores), np.std(scores)))
        resultados_multinomial_encoded.append(resultado_parcial)



A continuación se representan los datos obtenidos en una misma tabla.

In [20]:
resultados_sklearn = []
for r in resultados_multinomial, resultados_gaussian, resultados_categorical, resultados_multinomial_encoded:
    for x in r:
        resultados_sklearn.append(x)

df_sklearn = pd.DataFrame(resultados_sklearn, 
                  columns=['Error tic-tac-toe.data','Error german.data'], 
                  index=pd.MultiIndex.from_tuples([('Multinomial', 'Simple', 'No'), 
                                                   ('Multinomial', 'Simple', 'Si'), 
                                                   ('Multinomial', 'Cruzada', 'No'), 
                                                   ('Multinomial', 'Cruzada', 'Si'),
                                                   ('Gaussian', 'Simple', 'No'),  
                                                   ('Gaussian', 'Cruzada', 'No'), 
                                                   ('Categorical', 'Simple', 'No'), 
                                                   ('Categorical', 'Simple', 'Si'), 
                                                   ('Categorical', 'Cruzada', 'No'), 
                                                   ('Categorical', 'Cruzada', 'Si'),
                                                   ('MultinomialOneHot', 'Simple', 'No'), 
                                                   ('MultinomialOneHot', 'Simple', 'Si'), 
                                                   ('MultinomialOneHot', 'Cruzada', 'No'), 
                                                   ('MultinomialOneHot', 'Cruzada', 'Si'),], 
                                                  names=['Clasificador', 'Validación', 'Laplace']))

df_sklearn

Error tic-tac-toe.data  \
Clasificador      Validación Laplace                          
Multinomial       Simple     No       0.353125 +/- 0.011600   
                             Si       0.352083 +/- 0.029573   
                  Cruzada    No       0.343396 +/- 0.039223   
                             Si       0.343439 +/- 0.008916   
Gaussian          Simple     No       0.303125 +/- 0.022438   
                  Cruzada    No       0.281828 +/- 0.024779   
Categorical       Simple     No       0.304167 +/- 0.020989   
                             Si       0.291667 +/- 0.027362   
                  Cruzada    No       0.302683 +/- 0.018151   
                             Si       0.294361 +/- 0.020517   
MultinomialOneHot Simple     No       0.313542 +/- 0.038892   
                             Si       0.290625 +/- 0.036592   
                  Cruzada    No       0.296493 +/- 0.030618   
                             Si       0.301685 +/- 0.014866   

                                          Error german.data  
Clasificador      Validación Laplace                         
Multinomial       Simple     No       0.355000 +/- 0.022804  
                             Si       0.350000 +/- 0.026268  
                  Cruzada    No       0.360000 +/- 0.039875  
                             Si       0.360000 +/- 0.043932  
Gaussian          Simple     No       0.270000 +/- 0.038079  
                  Cruzada    No       0.266000 +/- 0.029732  
Categorical       Simple     No       0.326250 +/- 0.039745  
                             Si       0.267500 +/- 0.042205  
                  Cruzada    No       0.308750 +/- 0.016724  
                             Si       0.263333 +/- 0.015456  
MultinomialOneHot Simple     No       0.300000 +/- 0.046583  
                             Si       0.279000 +/- 0.015620  
                  Cruzada    No       0.298000 +/- 0.018868  
                             Si       0.269000 +/- 0.031686

### `tic-tac-toe.data`

En este conjunto de datos todos los clasificadores obtienen mejores resultados que MultinomialNB. CategoricalNB, por ejemplo, obtiene un 5% menos de error, lo cual tiene sentido considerando que todos los atributos del conjunto son categóricos. Aún así, sorprende también observar que GaussianNB obtiene un error muy similar a CategoricalNB (siendo el motivo principal que los atributos categóricos no tienen porque estar modelados correctamente con una distribución normal). De entre todos los clasificadores, el que obtiene los mejores resultados es MultinomialNB una vez se aplica OneHotEncoding a los datos. De nuevo, esto parece lógico ya que todos los atributos del conjunto de datos son categóricos y, por lo tanto, la aplicación de OneHotEncoding resulta muy eficiente.

Por lo general, aplicar Laplace obtiene mejores resultados. 

Por lo general, también, la validación cruzada obtiene mejores resultados que la validación simple.

### `german.data`

Al igual que en el anterior conjunto de datos, MultinomialNB es el peor clasificador de todos. En este caso, el que obtiene un menor error se trata de GaussianNB (teniendo en cuenta ambas validaciones). Esto podría tener sentido ya que 7 de los 20 atributos en el conjunto de datos son contínuos, por lo tanto su ajuste a través de una normal debería de ser mucho más eficiente que utilizar conteo por ejemplo. Además, como ya hemos visto con el conjunto `tic-tac-toe.data`, aunque haya atributos categóricos/nominales GaussianNB sigue haciendo un buen trabajo con las predicciones (dentro de lo posible). Sorprendentemente, MultinomialNB con OneHotEncoding también funciona bastante bien (quizás porque más de la mitad de los atributos son categóricos y algunos de los númericos no parecen distribuciones continuas sino cuentas o categóricos codificados como enteros). 

En todos los casos, aplicar Laplace obtiene un menor error.

Por lo general, la validación cruzada obtiene menor error que la validación simple. 

## 3. Conclusión

En el caso de `tic-tac-toe.data`, nuestra implementación de clasificador Naive Bayes, obtiene un error similar al del mejor clasificador de `scikit-learn` (MultinomialNB con OneHotEncoding). La diferencia es del orden de $0.1\%$. Para `german.data`, la mejora de nuestro clasificador es un poco más notable, alrededor del $1\%$. Esto tiene sentido ya que nuestro clasificador tiene una forma "personalizada" de tratar cada atributo, diferenciando si son nominales o continuos. En cambio, en este caso, GaussianNB trata a todos de la misma forma (tanto a los nominales como a lso continuos). Era de esperar, por lo tanto, que nuestro clasificador obtuviese un menor error. 